### Non-max Suppression

* NMS
Applied to remove redundant bounding boxes after predictions are made.

Before all predicted bounding boxes:
<img src="./ref_imgs/nms-before.png" alt="Description" width="100">

After nms to remove redundant bboxes
<img src="./ref_imgs/nms-after.png" alt="Description" width="100">


<b>What NMS is doing.</b>

0. Before we do NMS, we have a basic threshold this is not the IOU_threshold, this threshold is just to remove bboxes with a low confidence score. then do the following

1. Example: we get 3 bboxes 
<img src="./ref_imgs/during-nms.png" alt="Description" width="200">
    1. each of them have a probability score associated with that box 0.3, 0.9, 0.6, the probability is bewteen 0-1 and it indicts how likely that there is an object in that bbox.

2. We grab the highest bbox which in this case is the one with a 0.9 probability
<img src="./ref_imgs/nms-01.png" alt="Description" width="200">

3. Then we compare it with the other boxes that have a lower probability, and we calculate the IOU between them, example IOU = between 0.9 vs 0.6 bboxes = 0.51
<img src="./ref_imgs/nms-02.png" alt="Description" width="200">
    1. if the result is higher than an IOU_threshold, like 0.5 then we remove the 0.6 bbox. 

4. then we compare the 0.9 and 0.3 bboxes, example IOU result = 0.6, which is higher than out IOU_threshold, so we remove the 0.3 bbox

5. If we have multiple class predictions in the image, example: a car and a horse, then we need to do NMS for each different class separately.



In [1]:
import torch
from utils.IOU import intersection_over_union

In [ ]:
def non_max_suppression(
    bboxes, 
    IOU_threshold,
    prob_threshold,
    box_format="corners"):
    """ 
    Performs NMS
    
    Note:
        Non-Maximum Suppression (NMS) is a vital post-processing step in many computer vision tasks, particularly in object detection. It is used to refine the output of object detection models by eliminating redundant bounding boxes and ensuring that each object is detected only once.
    
    Parameters:
        bboxes (python:list) : predicted bounding boxes [ [1, 0.9, x1, y1, x2, y2], [etc..], [etc..], etc..]
            the 1 represents the class id, also is the index of which object it is from the dataset object list, example: 1 means its a car
            0.9 represents the probability
        
        IOU_threshold (float) : the iou threshold when comparing bounding boxes for NMS
        
        prob_threshold (float) : the threshold to remove bounding boxes with a low confidence score
    """
    
    assert type(bboxes) == list
    
    # remove bounding boxes with a low confidence score
    bboxes = [box for box in bboxes if box[1] > prob_threshold]
    
    # sort the bboxes with the highest probability at the beginning
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    
    bboxes_after_nms = []
    
    while bboxes:
        # grab a box from queue
        chosen_box = bboxes.pop(0)

        bboxes = [
            box for box in bboxes
                if box[0] != chosen_box[0] # check to see if the classes are the same if the bbox classes are different than we dont want to compare them IOU is only done when comparing bboxes for the same class, example : a car and a horse bbox
                or intersection_over_union(
                    torch.tensor(chosen_box[2:]), # just pass the coordinates from chosen box (x1, y1, x2, y2)
                    torch.tensor(box[2:]),
                    box_format=box_format
                )
                < IOU_threshold # if the IOU is less than the threshold then we will keep that box
        ]
        
        bboxes_after_nms.append(chosen_box)
        
    return bboxes_after_nms
        